In [29]:
# Manipulate the file system
import os
import shutil

# Copy dictionaries
import copy

# Regular expressions
import re

# Display errors in realtime
import ipywidgets as widgets

# work with dates
import datetime
import arrow
import time
import dateparser

# For scrubbing PII
import hashlib

# Convert stored string representation of a list to a list
import ast

# Recurse through a directory tree and return file names with glob
import glob

# Decode and re-encode mangled Arabic file names
import codecs

# Connect to a SQLite database in a lazy manner.
import sqlalchemy
import dataset

# Enables opening and reading of Excel files
import openpyxl

# Import Pandas for easy importing of an excel file
import csv

# Translating variables, sheet names, and workbook names from Arabic
# This is NOT free to use.
from google.cloud import translate

# Set the environment variable for the Google Service Account
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'C:\\Users\\clay\\Documents\\fxb-lcs-2b24f4f8a73a.json'

In [30]:
#If there's an existing db for this sheet, delete it
#so that we can copy from the template for a fresh start

try:
    os.remove("sams_data_phase12.sqlite")
    print("Removed template clone sams_data_phase12.sqlite")
except:
    pass

try:
    # Try to preserve a copy in case there is a problem and it has to be restored
    shutil.copy2("sams_data_phase12_template.sqlite","sams_data_phase12.sqlite")
    
    print("Created database from template: sams_data_phase12.sqlite")
except:
    pass

Removed template clone sams_data_phase12.sqlite
Created database from template: sams_data_phase12.sqlite


In [37]:
db = dataset.connect("sqlite:///sams_data_phase12.sqlite")

In [38]:
tab_files = db['files']

In [33]:
date_dict = {
    'jan':1,
    'Jan':1,
    'january':1,
    'January':1,
    "feb":2,
    'Feb':2,
    'february':2,
    'February':2,
    'mar':3,
    'Mar':3,
    'March':3,
    'march':3,
    'Apr':4,
    'apr':4,
    'April':4,
    'april':4,
    'May':5,
    'may':5,
    'Jun':6,
    'jun':6,
    'June':6,
    'june':6,
    'jul':7,
    'Jul':7,
    'July':7,
    'july':7,
    'Aug':8,
    'aug':8,
    'August':8,
    'august':8,
    'September':9,
    'september':9,
    'sep':9,
    'Sep':9,
    'Sept':9,
    'sept':9,
    'Oct':10,
    'oct':10,
    'October':10,
    'october':10,
    'Nov':11,
    'nov':11,
    'November':11,
    'november':11,
    'Dec':12,
    'dec':12,
    'December':12,
    'december':12
}

In [34]:
update_rows = []

for row in tab_files.find():
    row['month'] = None
    for m in date_dict.keys():
        if (m is not None and m in row['path']) or (m and row['translation'] and m in row['translation']):
            row['month'] = date_dict[m]
            break
    if row['month']:
        update_rows.append(row)

In [35]:
# This is mostly accurate. There appear to be < 10 errors in 1100 hits. I'll manually fix the rest
# in the database. Most errors were coded to March due to a Maraa facility that hit on Mar.
for ur in update_rows:
    tab_files.update(ur,['id'])

There were a few errors. I created a temp table, called `files_temp`, with:

```
CREATE TABLE files_temp AS SELECT id, path, translation, year, month FROM files;
```

The main reason was to isolate the columns I needed to see in order to tell if I made an error.

Now, I'm going to map the values from the `month` column back to the `month` column on the `files` table.

In [39]:
tab_files_temp = db['files_temp']

In [40]:
for row in tab_files_temp.find():
    newrec = {"id":row['id'],"month":row['month']}
    tab_files.update(newrec,['id'])

In [41]:
# Get rid of the temp table
tab_files_temp.drop()

True

Query to export some data for initial distribution analysis:

```
SELECT facilities.facilityname, facilities.country, facilities.governorate, facilities.district, facilities.subdistrict, facilities.facility_type, t.*
FROM facilities
JOIN (
	SELECT files.path, files.ungarbled, files.facility_id, files.year, files.month, count(full_raw_scrubbed.id) as num_records
	FROM files
	JOIN full_raw_scrubbed ON full_raw_scrubbed.a_file_id = files.id
	WHERE files.facility_id IS NOT NULL
	GROUP BY files.id, files.facility_id, files.year, files.month
	ORDER BY files.facility_id ASC, files.year ASC, files.month ASC, num_records DESC
) t ON facilities.id = t.facility_id;
```

In [42]:
# Copy the database over as the template for the next file.

# Do not rerun this cell!
shutil.copy2('sams_data_phase12.sqlite','sams_data_phase13_template.sqlite')

'sams_data_phase13_template.sqlite'